In [8]:
import os
import pandas as pd
import numpy as np
from collections import Counter

In [9]:
# --- 1. Carga y Preparación de Datos ---

# Definimos la ruta relativa a la carpeta que contiene los datasets
data_path = os.path.join('src', 'datasets', 'diginetica')

# Construimos las rutas completas para cada archivo
views_file = os.path.join(data_path, 'train-item-views.csv')
# categories_file = os.path.join(data_path, 'product-categories.csv') # No es necesario para estos modelos

try:
    # Cargar los datos desde la ruta especificada
    views_df = pd.read_csv(views_file, sep=';')
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta esperada: {views_file}")
    print("Asegúrate de que la estructura de carpetas coincida con la del árbol proporcionado.")
    exit()

# --- Preprocesamiento Adicional: Filtrado de Items y Sesiones ---
print(f"Datos originales: {len(views_df)} interacciones")

# 1. Filtrar ítems poco frecuentes
min_item_support = 5
item_counts = views_df['itemId'].value_counts()
items_to_keep = item_counts[item_counts >= min_item_support].index
views_df = views_df[views_df['itemId'].isin(items_to_keep)]
print(f"Datos tras filtrar items (< {min_item_support} apariciones): {len(views_df)} interacciones")

# 2. Filtrar sesiones cortas
min_session_length = 2
session_lengths = views_df['sessionId'].value_counts()
sessions_to_keep = session_lengths[session_lengths >= min_session_length].index
views_df = views_df[views_df['sessionId'].isin(sessions_to_keep)]
print(f"Datos tras filtrar sesiones (< {min_session_length} items): {len(views_df)} interacciones")


# Convertir a formato de fecha y ordenar
views_df['eventdate'] = pd.to_datetime(views_df['eventdate'])
views_df = views_df.sort_values(by=['sessionId', 'timeframe']).reset_index(drop=True)

# Agrupar por sesión para crear secuencias de ítems
sessions = views_df.groupby('sessionId')['itemId'].apply(list)

# Crear conjuntos de entrenamiento y prueba
train_sessions = []
test_targets = []

for session in sessions:
    # La comprobación de len > 1 es redundante por el filtro previo, pero es una buena práctica mantenerla
    if len(session) > 1:
        train_sessions.append(session[:-1])
        test_targets.append(session[-1])

print(f"\nTotal de sesiones procesadas: {len(sessions)}")
print(f"Sesiones para entrenamiento/prueba: {len(train_sessions)}")

Datos originales: 1235380 interacciones
Datos tras filtrar items (< 5 apariciones): 1095051 interacciones
Datos tras filtrar sesiones (< 2 items): 999198 interacciones

Total de sesiones procesadas: 205697
Sesiones para entrenamiento/prueba: 205697


In [10]:
# --- 2. Implementación de los Modelos de Recomendación ---

class RandomRecommender:
    """Recomienda ítems al azar."""
    def __init__(self):
        self.unique_items = []

    def fit(self, train_data):
        all_items = [item for session in train_data for item in session]
        self.unique_items = list(set(all_items))
        print("Modelo Aleatorio entrenado.")

    def predict(self, session_input, k=10):
        return np.random.choice(self.unique_items, size=k, replace=False).tolist()

class MostPopularRecommender:
    """Recomienda los ítems más populares."""
    def __init__(self):
        self.top_k_items = []

    def fit(self, train_data, k=10):
        all_items = [item for session in train_data for item in session]
        item_counts = Counter(all_items)
        self.top_k_items = [item for item, count in item_counts.most_common(k)]
        print("Modelo Más Popular entrenado.")

    def predict(self, session_input, k=10):
        return self.top_k_items

In [6]:
# --- 3. Implementación de las Métricas de Evaluación ---

def hits_at_k(recommendations, ground_truth, k):
    return 1 if ground_truth in recommendations[:k] else 0

def ndcg_at_k(recommendations, ground_truth, k):
    if ground_truth in recommendations[:k]:
        rank = recommendations.index(ground_truth) + 1
        return 1 / np.log2(rank + 1)
    return 0

def mrr_at_k(recommendations, ground_truth, k):
    if ground_truth in recommendations[:k]:
        rank = recommendations.index(ground_truth) + 1
        return 1 / rank
    return 0

In [11]:
K = 10

random_model = RandomRecommender()
popular_model = MostPopularRecommender()

random_model.fit(train_sessions)
popular_model.fit(train_sessions, k=K)

metrics_random = {'hits': [], 'ndcg': [], 'mrr': []}
metrics_popular = {'hits': [], 'ndcg': [], 'mrr': []}

for i, session in enumerate(train_sessions):
    target = test_targets[i]
    
    recs_random = random_model.predict(session, k=K)
    metrics_random['hits'].append(hits_at_k(recs_random, target, K))
    metrics_random['ndcg'].append(ndcg_at_k(recs_random, target, K))
    metrics_random['mrr'].append(mrr_at_k(recs_random, target, K))
    
    recs_popular = popular_model.predict(session, k=K)
    metrics_popular['hits'].append(hits_at_k(recs_popular, target, K))
    metrics_popular['ndcg'].append(ndcg_at_k(recs_popular, target, K))
    metrics_popular['mrr'].append(mrr_at_k(recs_popular, target, K))

avg_hits_random = np.mean(metrics_random['hits'])
avg_ndcg_random = np.mean(metrics_random['ndcg'])
avg_mrr_random = np.mean(metrics_random['mrr'])

avg_hits_popular = np.mean(metrics_popular['hits'])
avg_ndcg_popular = np.mean(metrics_popular['ndcg'])
avg_mrr_popular = np.mean(metrics_popular['mrr'])

print("\n--- Resultados de Evaluación (K=10) ---")
print("\nModelo: Random Recommender")
print(f"Hits@{K}:   {avg_hits_random * 100:.2f}")
print(f"NDCG@{K}:   {avg_ndcg_random * 100:.2f}")
print(f"MRR@{K}:    {avg_mrr_random * 100:.2f}")

print("\nModelo: Most Popular Recommender")
print(f"Hits@{K}:   {avg_hits_popular * 100:.2f}")
print(f"NDCG@{K}:   {avg_ndcg_popular * 100:.2f}")
print(f"MRR@{K}:    {avg_mrr_popular * 100:.2f}")

Modelo Aleatorio entrenado.
Modelo Más Popular entrenado.

--- Resultados de Evaluación (K=10) ---

Modelo: Random Recommender
Hits@10:   0.02
NDCG@10:   0.01
MRR@10:    0.01

Modelo: Most Popular Recommender
Hits@10:   0.68
NDCG@10:   0.31
MRR@10:    0.20
